In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from sklearn.decomposition import NMF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Configuration de l'affichage pour voir toutes les colonnes
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Librairies chargées avec succès !")

Librairies chargées avec succès !


In [5]:
# Chargement de la matrice et des catalogues
print("--- Chargement des données ---")
X = load_npz('../out/user_permission_matrix_sparse.npz')
users_df = pd.read_csv('../out/users_catalog.csv')
perms_df = pd.read_csv('../out/perm_catalog.csv')
apps_df = pd.read_csv('../out/app_catalog.csv')

print(f"Matrice chargée : {X.shape} (Utilisateurs x Permissions)")
print(f"Nombre d'utilisateurs : {len(users_df)}")
print(f"Nombre de permissions : {len(perms_df)}")

--- Chargement des données ---
Matrice chargée : (5000, 410) (Utilisateurs x Permissions)
Nombre d'utilisateurs : 5000
Nombre de permissions : 410


In [10]:
# --- Configuration NMF avec NETTOYAGE ---

# 1. Calcul de la fréquence des permissions
# On regarde combien d'utilisateurs possèdent chaque permission
# X est notre matrice (User x Perms)
# On convertit en binaire pour compter (au cas où il y aurait des valeurs > 1)
X_binary = (X > 0).astype(int)
user_count_per_perm = np.array(X_binary.sum(axis=0)).flatten()
total_users = X.shape[0]

# Fréquence (en %)
freq_per_perm = user_count_per_perm / total_users

# 2. Définition des bornes de filtrage
# On vire ce qui est trop commun (> 70% des gens l'ont) -> C'est du "Bruit de fond" (ex: Login, Email, Antivirus)
# On vire ce qui est trop rare (< 0.5% des gens l'ont) -> C'est du "Bruit rare"
UPPER_LIMIT = 0.70 
LOWER_LIMIT = 0.005 

# Masque des permissions à garder
perms_to_keep_mask = (freq_per_perm <= UPPER_LIMIT) & (freq_per_perm >= LOWER_LIMIT)

print(f"Permissions totales : {len(perms_df)}")
print(f"Permissions trop communes (> {UPPER_LIMIT*100}%) : {np.sum(freq_per_perm > UPPER_LIMIT)}")
print(f"Permissions trop rares (< {LOWER_LIMIT*100}%)   : {np.sum(freq_per_perm < LOWER_LIMIT)}")
print(f"Permissions gardées pour la NMF : {np.sum(perms_to_keep_mask)}")

# 3. On filtre la matrice X pour ne garder que les colonnes utiles
# C'est crucial : on ne change pas les lignes (users), juste les colonnes (perms)
X_filtered = X[:, perms_to_keep_mask]

# Il faut aussi mettre à jour notre catalogue de permissions pour qu'il corresponde aux nouvelles colonnes
perms_df_filtered = perms_df[perms_to_keep_mask].reset_index(drop=True)


# --- LANCEMENT DE LA NMF SUR LA MATRICE PROPRE ---
N_ROLES = 25 # On peut réduire un peu car on a enlevé le bruit
print(f"\nLancement de la NMF sur {X_filtered.shape[1]} permissions distinctives...")

model_nmf = NMF(n_components=N_ROLES, init='nndsvd', random_state=42, max_iter=1000)
W = model_nmf.fit_transform(X_filtered)
H = model_nmf.components_

# Normalisation
W_norm = W / W.sum(axis=1, keepdims=True)
W_norm = np.nan_to_num(W_norm)

print("NMF terminée sur données nettoyées !")

# --- IMPORTANT ---
# Pour la suite du code (Cellule 6 - Démo), on doit utiliser perms_df_filtered
# au lieu de perms_df pour afficher les bons noms !
perms_df = perms_df_filtered

Permissions totales : 410
Permissions trop communes (> 70.0%) : 0
Permissions trop rares (< 0.5%)   : 314
Permissions gardées pour la NMF : 96

Lancement de la NMF sur 96 permissions distinctives...
NMF terminée sur données nettoyées !


In [11]:
# --- DÉFINITION DU SEUIL ---
# Si un utilisateur a plus de 10% de son "ADN" dans un rôle, on considère qu'il l'a.
THRESHOLD = 0.10

# Y_binary est une matrice de 0 et 1.
# Chaque ligne correspond à un utilisateur, chaque colonne à un rôle.
# Exemple : [1, 0, 0, 1, ...] signifie "Possède le Rôle 0 et le Rôle 3"
Y_binary = (W_norm > THRESHOLD).astype(int)

# Vérification statistique
avg_roles = np.mean(np.sum(Y_binary, axis=1))
print(f"--- Statistiques Multi-Rôles ---")
print(f"Avec un seuil de {THRESHOLD*100}%, un utilisateur possède en moyenne {avg_roles:.2f} rôles.")
print("Si ce chiffre est proche de 1, baisse le seuil. S'il est > 5, monte le seuil.")

--- Statistiques Multi-Rôles ---
Avec un seuil de 10.0%, un utilisateur possède en moyenne 2.74 rôles.
Si ce chiffre est proche de 1, baisse le seuil. S'il est > 5, monte le seuil.


In [12]:
print("--- Préparation et Entraînement Supervisé ---")

# 1. Encodage des entrées (X)
le_dept = LabelEncoder()
users_df['dept_encoded'] = le_dept.fit_transform(users_df['department'])

le_pos = LabelEncoder()
users_df['pos_encoded'] = le_pos.fit_transform(users_df['position'])

le_loc = LabelEncoder()
users_df['loc_encoded'] = le_loc.fit_transform(users_df['location'])

X_features = users_df[['dept_encoded', 'pos_encoded', 'loc_encoded']]

# 2. Séparation Train / Test
X_train, X_test, y_train, y_test = train_test_split(X_features, Y_binary, test_size=0.2, random_state=42)

# 3. Entraînement
# Random Forest gère nativement le "Multi-Output" (prédire plusieurs 1 à la fois)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 4. Évaluation rapide
y_pred = clf.predict(X_test)
# L'accuracy exacte est sévère (il faut avoir TOUT bon sur la ligne), donc on regarde un score moyen
print(f"Modèle entraîné sur {len(X_train)} utilisateurs.")
print(f"Précision 'Subset' (Tout bon ou rien) : {accuracy_score(y_test, y_pred):.2%}")

--- Préparation et Entraînement Supervisé ---
Modèle entraîné sur 4000 utilisateurs.
Précision 'Subset' (Tout bon ou rien) : 47.40%


In [13]:
def describe_role(role_id):
    """Fonction helper pour donner un nom lisible à un rôle ID"""
    # On regarde les 3 apps principales du rôle pour comprendre ce que c'est
    top_idx = H[role_id].argsort()[::-1][:3]
    top_apps = perms_df.iloc[top_idx].merge(apps_df, on='application_id')['app_name'].unique()
    return f"Rôle {role_id} (ex: {', '.join(top_apps)})"

print("--- SIMULATION : PROVISIONING AUTOMATIQUE ---")

# --- PARAMÈTRES DU NOUVEAU VENU ---
NEW_DEPT = "Department Sales"
NEW_POS  = "Sales Manager"
NEW_LOC  = "Lyon"

print(f"Arrivée d'un : {NEW_POS} / {NEW_DEPT} / {NEW_LOC}")

try:
    # 1. Encodage des infos
    encoded_input = [[
        le_dept.transform([NEW_DEPT])[0],
        le_pos.transform([NEW_POS])[0],
        le_loc.transform([NEW_LOC])[0]
    ]]
    
    # 2. Prédiction
    predicted_vector = clf.predict(encoded_input)[0] # Renvoie [0, 1, 0, 1, ...]
    
    # 3. Interprétation
    # On récupère les indices où il y a un '1'
    found_role_indices = np.where(predicted_vector == 1)[0]
    
    print(f"\n✅ L'IA suggère d'attribuer {len(found_role_indices)} rôles :")
    for r_id in found_role_indices:
        print(f"  -> {describe_role(r_id)}")

except Exception as e:
    print(f"Erreur : Une des valeurs (Ville/Poste) n'est pas connue du modèle entrainé. ({e})")

--- SIMULATION : PROVISIONING AUTOMATIQUE ---
Arrivée d'un : Sales Manager / Department Sales / Lyon

✅ L'IA suggère d'attribuer 2 rôles :
  -> Rôle 4 (ex: CrowdStrike Falcon, Access Badge System - Toulouse)
  -> Rôle 12 (ex: Access Badge System - Lyon, Zoom)


/Users/tidianetall/Desktop/Desktop/Cours/PCE/Wavestone/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
